# Assignment Set 2

*Authors*: Myriam Belkhatir, Salomé Poulain, Shania Sinha

## Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
from matplotlib.animation import FuncAnimation, PillowWriter
from importlib import reload
import glob
import os
import imageio

%matplotlib inline

## A. Growth Model DLA

In [ ]:
import src.dla
reload(src.dla)
from src.dla import DLASimulation

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Set grid size
grid_size_100 = 100

# Create and run standard DLA simulation (η=1.0)
dla = DLASimulation(N=grid_size_100, eta=1.0)

# Uncomment to run the simulation
dla.run_simulation()

Not implemented: Make a gif of the resulting plots.

In [ ]:
# # Create the output directory if it doesn't exist
# os.makedirs("results", exist_ok=True)

# # Path to images
# image_path = "results/dla_output_eta1.0/*.png"
# image_files = sorted(glob.glob(image_path))

# # Check if any images are found
# if not image_files:
#     print(f"No image files found at {image_path}")
# else:
#     # Read images
#     images = [imageio.imread(file) for file in image_files]
    
#     # Create GIF
#     output_file = "results/dla_output_eta1.gif"
#     imageio.mimsave(output_file, images, duration=0.2)  # duration is in seconds per frame
#     print(f"GIF saved as {output_file}")

In [ ]:
def run_eta_study(eta_values=[0.5, 1.0, 2.0], N=100, num_particles=1000):
    """
    Run multiple DLA simulations with different eta values.
    """
    for eta in eta_values:
        print(f"Running simulation with η={eta}")
        output_dir = f"results/dla_output_eta{eta:.1f}"
        
        # Set random seed for reproducibility
        np.random.seed(42)
        
        # Create and run DLA simulation
        dla = DLASimulation(N=N, eta=eta, output_dir=output_dir)
        dla.run_simulation()

In [ ]:
# Investigate the influence of eta parameter
testing_etas = [0.85, 1.5]
run_eta_study(eta_values=testing_etas, N=grid_size_100)

### A.1. Can we optimize further?
Analyzing effect of $\omega$ and grid size on the cluster formation.

In [ ]:
import src.analysis_dla
reload(src.analysis_dla)
from src.analysis_dla import *

In [ ]:
# Part 1: Analyze omega performance for a fixed grid size
omega_values, total_iterations, execution_times = analyze_omega_performance(N=grid_size_100, num_particles=1000)

# Plot the results
plot_results(omega_values, total_iterations, execution_times, grid_size_100)

In [ ]:
# Part 2: Find optimal omega for different grid sizes
grid_sizes = [10, 50, 150, 200]

N_values, optimal_omegas, theoretical_omegas = find_optimal_omega_vs_gridsize()

# Plot optimal omega vs grid size
plot_optimal_omega_vs_N(N_values, optimal_omegas, theoretical_omegas)

## B. and C. Monte-Carlo DLA Simulation

In [ ]:
import src.monte_carlo_dla
reload(src.monte_carlo_dla)
from src.monte_carlo_dla import RandomWalker

In [ ]:
# Run simulation
stochastic_dla = RandomWalker(N=100, num_particles=1000)
stochastic_dla.run_simulation()

## D. Gray-Scott model

In [ ]:
import src.gray_scott
reload(src.gray_scott)
from src.gray_scott import GrayScottModel

In [ ]:
# Initialize the Gray-Scott model
N = 200
steps = 15000
Du = 0.16
Dv = 0.08

### Explore broad range of f and k

In [ ]:
# Define parameter ranges from low to high
F_grid_extended = np.linspace(0.03, 0.14, 3) 
k_grid_extended = np.linspace(0.045, 0.065, 3)

# Create meshgrid - switch order to get F on y-axis, k on x-axis
k_mesh_ext, F_mesh_ext = np.meshgrid(k_grid_extended, F_grid_extended)
F_points_ext = F_mesh_ext.flatten()
k_points_ext = k_mesh_ext.flatten()

# Create descriptions
descriptions = [f"F={f:.3f}, k={k:.3f}" for f, k in zip(F_points_ext, k_points_ext)]

# Create models for each parameter set
models = []
for f, k, desc in zip(F_points_ext, k_points_ext, descriptions):
    model = GrayScottModel(N=N, Du=Du, Dv=Dv, f=f, k=k)
    models.append(model)

In [ ]:
# Run simulations
for i, model in enumerate(models):
    model.run(steps=steps, save_interval=50)

In [ ]:
# Create a 3x3 plot of the final U field
fig, axs = plt.subplots(3, 3, figsize=(10, 10), constrained_layout=True, sharex=True, sharey=True)

for i, model in enumerate(models):
    u_final, _ = model.simulation[-1] 
    row = 2 - (i // 3)  # Reverse row index for bottom-up ordering
    col = i % 3        
    ax = axs[row, col]  
    im = ax.imshow(u_final, cmap="viridis", vmin=0, vmax=1)
    ax.set_title(f"{descriptions[i]}", fontsize=18)  
    ax.axis("off")

# Add a shared colorbar with proper font size configuration
cbar = fig.colorbar(im, ax=axs, orientation='vertical', fraction=0.046, pad=0.04)
cbar.ax.tick_params(labelsize=18) 

plt.savefig("results/gray_scott/gray_scott_3x3_final_states.pgf")
plt.show()

### Explore zoomed in region with different f and k values for distinct patterns

In [ ]:
# Format: (Du, Dv, f, k, description)
# Parameters from Har-Shemesh et al. (2015)
parameter_sets = [
    (0.0416, 0.0625),  
    (0.0392, 0.0649),  
    (0.0175, 0.0504),  
    (0.0295, 0.0561)  
]

# Complete the parameter sets with Du, Dv, and descriptions
descriptions = ["Worm-like structures", "Maze-like patterns", "Transition zone", "Edge of pattern-forming region"]
complete_params = [(Du, Dv, f, k, desc) for (f, k), desc in zip(parameter_sets, descriptions)]

# Create the models for each set of parameters
models = []
for Du, Dv, f, k, desc in complete_params:
    model = GrayScottModel(N=N, Du=Du, Dv=Dv, f=f, k=k)
    models.append(model)

In [ ]:
# Run simulations
for i, model in enumerate(models):
    model.run(steps=steps, save_interval=50)

In [ ]:
# Create a 2x2 plot of the final U field from the stored simulation snapshots
fig, axs = plt.subplots(2, 2, figsize=(10, 10), constrained_layout=True)

for i, model in enumerate(models):
    # Get the final snapshot stored in the simulation attribute (u, v)
    u_final, _ = model.simulation[-1]
    ax = axs[i // 2, i % 2]
    im = ax.imshow(u_final, cmap="viridis", vmin=0, vmax=1)
    
    # Extract f and k values from parameter_sets for the title
    f_val = parameter_sets[i][0]
    k_val = parameter_sets[i][1]
    ax.set_title(f"F={f_val:.4f}, k={k_val:.4f}", fontsize=18)
    ax.axis("off")

# Add a single colorbar for the entire figure
cbar = fig.colorbar(im, ax=axs, orientation='vertical', fraction=0.046, pad=0.04)
cbar.ax.tick_params(labelsize=18)

plt.savefig("results/gray_scott/gray_scott_2x2_final_states.pgf")
plt.show()

In [ ]:
# Create and save GIF animations for each simulation
for i, model in enumerate(models):
    ani = model.create_animation(interval=50)
    
    # Use the complete_params list which contains descriptions
    description = parameter_sets[i]
    filename = f"results/gray_scott/gif_{description}.gif"
    
    print(f"Saving animation as {filename} ...")
    ani.save(filename, writer=PillowWriter(fps=20))

# References

Har-Shemesh et al., 2015
"Information geometric analysis of phase transitions in complex patterns: the case of the Gray-Scott reaction-diffusion model",
J. Stat. Mech. Theory Exp., 2016(04), 043301.
DOI: 10.1088/1742-5468/2016/04/043301